In [1]:
import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from sklearn.svm import LinearSVC

train_data = np.loadtxt("train.dat")
test_data = np.loadtxt("test.dat")
x_train = train_data[:, :-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]


def my_map(x):
 x_t = np.ones((x.shape[0],x.shape[1]))
 for n in range(x.shape[0]):
  for i in range(x.shape[1]):
   for c in x[n,i:32]:
    x_t[n][i] = x_t[n][i] * (1-2*c)
 xxT = np.zeros((x_t.shape[0],x_t.shape[1],x_t.shape[1]))
 xxTu = []
 xxtu = []
 for i in range(x_t.shape[0]):
    temp = x_t[i].reshape(32,1)
    xxT[i] = np.multiply(temp,temp.T)
    for r in range(32):
      for c in range(r+1,32):
        xxtu.append(xxT[i][r][c])
    xxTu.append(xxtu)
    xxtu = []
 xxTu= np.array(xxTu)
 x_mapped = np.concatenate((xxTu,x_t),axis=1)
 return x_mapped

def my_fit(x_train,y_train):
  C_value = 11
  max_iter_value = 10000
  tol_value = 0.001

  model_SVC = LinearSVC(loss='hinge', tol=tol_value, C=C_value, max_iter=max_iter_value)
  x = my_map(x_train)
  y = y_train

  model_SVC.fit(x,y)

  w = model_SVC.coef_.flatten()
  b = model_SVC.intercept_

  return w,b

In [2]:
import time as tm

In [3]:
Z_trn = np.loadtxt( "secret_train.dat" )
Z_tst = np.loadtxt( "secret_test.dat" )

n_trials = 5

d_size = 0
t_train = 0
t_map = 0
acc = 0

In [5]:
for t in range( n_trials ):
	tic = tm.perf_counter()
	w, b = my_fit( Z_trn[:, :-1], Z_trn[:,-1] )
	toc = tm.perf_counter()
	t_train += toc - tic

	d_size += w.shape[0]

	tic = tm.perf_counter()
	feat = my_map( Z_tst[:, :-1] )
	toc = tm.perf_counter()
	t_map += toc - tic

	scores = feat.dot( w ) + b
	pred = np.zeros_like( scores )
	pred[scores > 0] = 1
	acc += np.average( Z_tst[ :, -1 ] == pred )

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [6]:
d_size /= n_trials
t_train /= n_trials
t_map /= n_trials
acc /= n_trials

print( d_size, t_train, t_map, 1 - acc )

633.6 86.86423324500001 10.31761798179998 0.00782000000000005
